In [151]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint


from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

from src.UVMappingPipeline import UVMappingPipeline
from utils.sys_utils import *
from utils.seg_utils import *
from utils.uv_mapping_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pipe = UVMappingPipeline()

In [ ]:
# tdir = "/home/hunter/data/images/obstacles"
tdir = "/home/hunter/data/images/test_cases"
# tdir = "/home/hyoung/data/images/obstacles"
# tdir = homeDir + "/test/iai/obstacles"

found_imgs, _ = grab_dir_images(tdir)

In [ ]:
%reload_ext autoreload
plt.close('all')

p = interactive(pipe.pipeline,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    line_method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThreshU = (0,255,1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4], p.children[9]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))

row5 = HBox([p.children[10], p.children[11], p.children[12], ], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[16]], layout = Layout(flex_flow='row wrap'))
row7 = HBox([p.children[13], p.children[14], p.children[15]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6,row7], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

In [ ]:
tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

normImg = (tmp1 - np.mean(tmp1))/np.std(tmp1)

dmax_norm = np.max(normImg)
dmin_norm = np.min(normImg)
test = cv2.normalize(normImg,None,0,1,cv2.NORM_MINMAX)
m = 255.0/(dmax_norm-dmin_norm)
print(dmin_norm,dmax_norm,m)
tmp = cv2.convertScaleAbs(test, test)#, m)#,np.mean(tmp))
print(np.min(tmp),np.max(tmp),np.mean(tmp))

fig1 = plt.figure(6)
fig1.canvas.set_window_title('Custom Filter Input')
plt.imshow(pipe.vmap,cmap='gray')
plt.show()

iters = 3; kSz = (1,6)
thresh = [5, 10, 20, 30]
filters = [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC+cv2.THRESH_BINARY_INV,cv2.THRESH_TOZERO]
kernelC = cv2.getStructuringElement(cv2.MORPH_CROSS,kSz)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))

_, grey = cv2.threshold(tmp1,5,255,filters[0])#+cv2.THRESH_TRUNC)

# gradient = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, kSz)
gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)

# tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
# preMorph = cv2.erode(grey,kernelE,iterations = 3)
# preMorph = cv2.dilate(grey,tmpKernel,iterations = 3)


# # morph = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,(1,2), iterations = 2)
# morph = cv2.morphologyEx(grey,cv2.MORPH_OPEN,(2,3), iterations = 1)

# _,preMorph = cv2.threshold(preMorph,1,255,cv2.THRESH_BINARY_INV)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,15))
# preMorph = cv2.dilate(preMorph,tmpKernel,iterations = 3)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,3))
# preMorph = cv2.erode(preMorph,kernelE,iterations = 2)
# print(preMorph.shape)

# mask = cv2.cvtColor(preMorph,cv2.COLOR_BGR2GRAY)
# res = cv2.bitwise_and(tmp,tmp, mask=mask)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

fig2 = plt.figure(7)
fig2.canvas.set_window_title('Custom Filter Output')
plt.imshow(out,cmap='gray')
plt.show()

In [ ]:
print("""
POTENTIALLY GOOD VMAP FILTERING
""")

tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

kSz = (1,6)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))
_, grey = cv2.threshold(tmp1,5,255,cv2.THRESH_BINARY)

gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

In [ ]:
# print(pipe.disparityBounds[obsId],pipe.xBounds[obsId])

tmp = np.copy(pipe.greyV)
tmp2 = np.copy(pipe.img)
tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
nObs = len(pipe.disparityBounds)
print(nObs)
b = 2; d = np.ceil(nObs / float(b))

plt.close(7)
fig, axes = plt.subplots(nrows=b, ncols=int(d), num=7)
obstacleImgs = []
for i in range(nObs):
    tmp2 = np.copy(pipe.img)
    tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
    try:
        xs = pipe.xBounds[i]
        ys,disp = pipe.find_obstacles(tmp,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True)
        cv2.rectangle(tmp2,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
    except:
        pass
    fig.axes[i].imshow(disp)
    obstacleImgs.append(tmp2)
plt.show()

In [ ]:
im = np.copy(pipe.img)
norm = (im - np.mean(im))/np.std(im)
h, w = norm.shape[:2]

umapOrig, vmapOrig,_ = pipe.get_uv_map(im,get_overlay=False)

plt.figure(0)
plt.imshow(im, cmap='gray')
plt.show()
plt.figure(1)
plt.imshow(norm, cmap='gray')
plt.show()

In [90]:
found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
_imgs = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    )

display(_imgs)

U2VsZWN0KGRlc2NyaXB0aW9uPXUnRmlsZTonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJScpLCBvcHRpb25zPSgnL2hvbWUvaHVudGVyL2RhdGEvaW1hZ2VzL3Rlc3RfY2FzZXMvZnJhbWXigKY=


In [286]:
from utils.uv_mapping_utils import *

_img = _imgs.value
img = cv2.imread(_img, cv2.IMREAD_GRAYSCALE)

umap, vmapMasked, img, obs, nObs, pxls,vmap,mask, maskInv, dt = uv_pipeline(_img, flag_displays=True)



# mask, maskInv,mPxls, ground_wins,_ = get_vmap_mask(newV, maxStep = 15)
tmpVmap = cv2.bitwise_and(vmap,vmap,mask=mask)

# nonzero = tmpVmap.nonzero()
# nonzeroy = np.array(nonzero[0])
# nonzerox = np.array(nonzero[1])

# print(nonzerox.shape)

plt.figure(9)
plt.imshow(mask)
plt.show()

# vs = cv2.approxPolyDP(pxls,3,0)
# cv2.polylines(tmpVmap, [vs], 0, (255,0,255),20)

# vx, vy, cx, cy = cv2.fitLine(pxls,cv2.DIST_L2,0, 0.01, 0.01)
# cv2.line(tmpVmap, (int(cx-vx*w), int(cy-vy*w)), (int(cx+vx*w), int(cy+vy*w)), (0, 0, 255))

# plt.figure(10)
# plt.imshow(tmpVmap)
# plt.show()

[INFO] Beginning Obstacle Search....
	[uv_pipeline] --- Took 0.024178 seconds to complete


In [265]:
h,w = vmap.shape[:2]
# black = np.zeros((h,w,3),dtype=np.uint8)
black = np.copy(vmap)
black = cv2.cvtColor(black,cv2.COLOR_GRAY2BGR)

print(pxls[-1,0],pxls)

# ploty = np.linspace(pxls[0,1], pxls[-1,1])
plotx = np.linspace(pxls[0,0], pxls[-1,0])
try: # Try fitting polynomial nonzero data
    fit = np.poly1d(np.polyfit(pxls[:,0],pxls[:,1], 2))
    # Generate x and y values for plotting
#     plotx = fit(ploty)
    ploty = fit(plotx)
except:
    print("ERROR: Function 'polyfit' failed!")
    fit = [0, 0]; plotx = [0, 0]
# xs = np.asarray(ploty,dtype=np.int32)
# ys = np.asarray(plotx,dtype=np.int32)

xs = np.asarray(plotx,dtype=np.int32)
ys = np.asarray(ploty,dtype=np.int32)

pts = np.vstack(([xs],[ys])).transpose()
cv2.polylines(black, [pts], 0, (255,0,255),5)
plt.figure(9)
plt.imshow(black)
# plt.plot(pxls[:,0],pxls[:,1])
# plt.plot(ploty,-plotx)

plt.show()

(34, array([[  3, 459],
       [  3, 448],
       [  3, 437],
       [  3, 425],
       [  3, 414],
       [  4, 403],
       [  4, 392],
       [  4, 381],
       [  5, 370],
       [  6, 359],
       [  7, 347],
       [  9, 336],
       [ 11, 325],
       [ 14, 314],
       [ 22, 302],
       [ 34, 293]]))


In [74]:
def tester2(_img, e1=1, e2=5,tmpThresh=3,greyThreshU=7,greyThreshV=5,cnt_thresh=100.0,
                 show_helpers=True,use_umap=True, flip_thresh_bin=True, norm_grey_thresh=0.5):
    img = cv2.imread(_img, cv2.IMREAD_GRAYSCALE)
    pipe.img = img
    plt.figure(0)
    plt.imshow(img)
    plt.show()

    if(flip_thresh_bin):
        umap, vmap, overlay = pipe.get_uv_map(img, get_overlay=True)
        dead_x = 2; dead_y = 10

        cv2.rectangle(umap,(0,0),(umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
        cv2.rectangle(umap,(0,umap.shape[0]-dead_y),(umap.shape[1],umap.shape[0]),(0,0,0), cv2.FILLED)

        cv2.rectangle(vmap,(0,0),(dead_x, vmap.shape[0]),(0,0,0), cv2.FILLED)
        cv2.rectangle(vmap,(vmap.shape[1]-dead_x,0),(vmap.shape[1],vmap.shape[0]),(0,0,0), cv2.FILLED)
    else:
        umap = umapping(img)
        vmap = vmapping(img)
        overlay = make_uv_overlay(img,umap,vmap)


    h,w = umap.shape[:2]
    clone = np.copy(umap)
#     cv2.rectangle(clone,(0,0),(w,10),(0,0,0), cv2.FILLED)
#     cv2.rectangle(clone,(0,h-10),(w,h),(0,0,0), cv2.FILLED)
    print(np.max(clone), np.min(clone))



    _,tmp = cv2.threshold(clone,greyThreshU,np.max(clone),cv2.THRESH_BINARY)
    opening = tmp
#     _,tmp = cv2.threshold(clone,greyThreshU,np.max(clone),cv2.THRESH_TOZERO)

#     cv2.rectangle(tmp,(0,0),(w,10),(0,0,0), cv2.FILLED)
#     cv2.rectangle(tmp,(0,h-10),(w,h),(0,0,0), cv2.FILLED)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(e1,e2))
#     opening = cv2.morphologyEx(tmp, cv2.MORPH_CLOSE, kernel)
#     _,opening = cv2.threshold(opening,tmpThresh,np.max(clone),cv2.THRESH_BINARY)

    _,greyV = cv2.threshold(vmap,greyThreshV,np.max(vmap),cv2.THRESH_BINARY)

    _, contours, hierarchy = cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    perimeters = [cv2.arcLength(cnt,True) for cnt in contours]

    filtered_cnts = [cnt for cnt in contours if cv2.arcLength(cnt,True) >= cnt_thresh]
    filtered_perimeters = [cv2.arcLength(cnt,True) for cnt in filtered_cnts]
    print("Filtered Contour Perimeters:",filtered_perimeters)

    display = cv2.cvtColor(opening, cv2.COLOR_GRAY2BGR)
    for cnt in filtered_cnts:
        cv2.drawContours(display, [cnt], 0, (255,0,0), 2)

    cnts, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds(filtered_cnts)

    nObs = len(pipe.disparityBounds)
    obsImgs = []
    disp = np.copy(img)
    display_obstacles = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)

    for i in range(nObs):
        tmp = np.copy(img)
        tmp = cv2.cvtColor(tmp,cv2.COLOR_GRAY2BGR)
        try:
            xs = pipe.xBounds[i]
            ys,disp = pipe.find_obstacles(greyV,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True, display=greyV)
            cv2.rectangle(tmp,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
            cv2.rectangle(display_obstacles,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
        except: pass
        obsImgs.append(tmp)


    plt.figure(1)
    plt.imshow(opening)
    plt.show()

    plt.figure(2)
    plt.imshow(overlay)
    plt.show()

    plt.figure(3)
    plt.imshow(display)
    plt.show()

    plt.figure(4)
    plt.imshow(display_obstacles)
    plt.show()

    plt.figure(6)
    plt.imshow(greyV)
    plt.show()

In [295]:
def thresholding(_img, e1=1, e2=5,tmpThresh=3,greyThreshU=7,greyThreshV=5,cnt_thresh=100.0,
                 show_helpers=True,use_umap=True, flip_thresh_bin=True, norm_grey_thresh=0.5
):
    if(show_helpers):
        umap, vmap, img, obs, nObs, pxls,_,_,_, dt = uv_pipeline(_img, flag_displays=flip_thresh_bin)
        display_obstacles = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        for ob in obs:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)
        plt.figure(0)
        plt.imshow(display_obstacles)
        plt.show()
    else:
        tester2(_img,e1,e2,tmpThresh,greyThreshU,greyThreshV,cnt_thresh,show_helpers,use_umap,flip_thresh_bin, norm_grey_thresh)
    
        

In [308]:
found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
ttt = interactive(thresholding,_img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ), e1 = (1,255,1), e2 = (1,255,1),
    tmpThresh = (0,255,1), greyThreshU = (0.0,255.0,0.1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)
display(ttt)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFNlbGVjdChkZXNjcmlwdGlvbj11J0ZpbGU6JywgbGF5b3V0PUxheW91dCh3aWR0aD11JzEwMCUnKSwgb3B0aW9ucz0oJy9ob21lL2h1bnRlci9kYXRhL2nigKY=


In [193]:
print(1/0.069936)
print(1/0.028278)

14.2987874628
35.3631798571
